<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-notebook-and-import-libraries" data-toc-modified-id="Create-notebook-and-import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create notebook and import libraries</a></span></li><li><span><a href="#Scraping-data-from-Wikipedia" data-toc-modified-id="Scraping-data-from-Wikipedia-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scraping data from Wikipedia</a></span></li><li><span><a href="#Cleaning-dataframe" data-toc-modified-id="Cleaning-dataframe-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cleaning dataframe</a></span></li><li><span><a href="#In-the-last-cell-of-your-notebook,-use-the-.shape-method-to-print-the-number-of-rows-of-your-dataframe." data-toc-modified-id="In-the-last-cell-of-your-notebook,-use-the-.shape-method-to-print-the-number-of-rows-of-your-dataframe.-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</a></span></li><li><span><a href="#Getting-geo-data-for-postcodes" data-toc-modified-id="Getting-geo-data-for-postcodes-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Getting geo data for postcodes</a></span></li><li><span><a href="#Explore-and-cluster-the-neighborhoods-in-Toronto." data-toc-modified-id="Explore-and-cluster-the-neighborhoods-in-Toronto.-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Explore and cluster the neighborhoods in Toronto.</a></span><ul class="toc-item"><li><span><a href="#Use-geopy-library-to-get-the-latitude-and-longitude-values-of-Toronto" data-toc-modified-id="Use-geopy-library-to-get-the-latitude-and-longitude-values-of-Toronto-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Use geopy library to get the latitude and longitude values of Toronto</a></span></li><li><span><a href="#Central-Toronto-folium-map" data-toc-modified-id="Central-Toronto-folium-map-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Central Toronto folium map</a></span></li><li><span><a href="#Exploring-Neighborhoods-in-Central-Toronto" data-toc-modified-id="Exploring-Neighborhoods-in-Central-Toronto-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Exploring Neighborhoods in Central Toronto</a></span></li><li><span><a href="#Analyse-Neighborhoods-in-Central-Toronto" data-toc-modified-id="Analyse-Neighborhoods-in-Central-Toronto-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Analyse Neighborhoods in Central Toronto</a></span></li><li><span><a href="#Cluster-Central-Toronto-neighbourhoods" data-toc-modified-id="Cluster-Central-Toronto-neighbourhoods-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Cluster Central Toronto neighbourhoods</a></span></li><li><span><a href="#Examine-clusters" data-toc-modified-id="Examine-clusters-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>Examine clusters</a></span></li></ul></li></ul></div>

# Create notebook and import libraries

In [140]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [141]:
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

# Scraping data from Wikipedia

In [142]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [143]:
x = BeautifulSoup(req.text,'lxml')
souptable = x.find('table')
souptable

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [144]:
headings = []
for row in souptable.find_all('th'):
    headings.append(row.text.strip('\n'))
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


In [145]:
content = []
for row in souptable.find_all('tr'):
    if not row.find_all('th'):
        content.append(row.text.strip('\n').split('\n\n'))
print(content)

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'], ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'], ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'], ['M1B', 'Scarborough', 'Malvern, Rouge'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills'], ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'], ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'], ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'], ['M2C', 'Not assigned', 'N

In [146]:
toronto = pd.DataFrame(content, columns = headings)
toronto

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Cleaning dataframe

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [147]:
#rename columns names for convenience, lower case and not spaces
toronto.set_axis(['postal_code','borough','neighbourhood'], axis = 'columns', inplace = True)
toronto

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [149]:
#check that not assigned boroughs are same as neighbourhood.
print(toronto[(toronto['neighbourhood'] != 'Not assigned') & (toronto['borough'] == 'Not assigned')])
print(toronto[(toronto['neighbourhood'] == 'Not assigned') & (toronto['borough'] != 'Not assigned')])

Empty DataFrame
Columns: [postal_code, borough, neighbourhood]
Index: []
Empty DataFrame
Columns: [postal_code, borough, neighbourhood]
Index: []


In [111]:
#let's drop all not assigned rows
toronto = toronto[toronto['borough'] != 'Not assigned']
toronto

,postal_code,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [112]:
#check that df already has combined boroughs by postal code
toronto[toronto['postal_code'] == 'M5A']

,postal_code,borough,neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

This check was done previously, all not assigned neighborhoods are identical to not assigned boroughs.

In [113]:
#check that there are no not assigned neighborhoods
toronto[toronto['neighbourhood'] == 'Not assigned']

,postal_code,borough,neighbourhood


# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [114]:
toronto.shape

(103, 3)

# Getting geo data for postcodes

In [115]:
#import file with geo data
geodata = pd.read_csv(r'C:\Users\Inga\Desktop\IBM Course\IBM Notebooks\9_Capstone\Geospatial_Coordinates.csv')
geodata.set_axis(['postal_code','latitude', 'longitude'], axis = 'columns', inplace = True)
geodata

,postal_code,latitude,longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [116]:
#merge geodata with toronto df
toronto = toronto.merge(geodata, on='postal_code', how = 'left')

In [117]:
#check there are no nulls or nans
toronto.info()
toronto.isna().count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   postal_code    103 non-null    object 
 1   borough        103 non-null    object 
 2   neighbourhood  103 non-null    object 
 3   latitude       103 non-null    float64
 4   longitude      103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


postal_code      103
borough          103
neighbourhood    103
latitude         103
longitude        103
dtype: int64

# Explore and cluster the neighborhoods in Toronto. 

## Use geopy library to get the latitude and longitude values of Toronto

In [118]:
#adding toronto geo data
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [119]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['latitude'], toronto['longitude'], 
                                           toronto['borough'], toronto['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Central Toronto folium map

As suggested in the task let's analyse only Central Toronto

In [120]:
# create df only with Central Toronto borough
central_toronto = toronto[toronto['borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto

,postal_code,borough,neighbourhood,latitude,longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [121]:
#idenitify Central Toronto geo data
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="ct_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6708625, -79.37279241253721.


In [122]:
# create map of Central Toronto using latitude and longitude values
map_centraltoronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(central_toronto['latitude'], central_toronto['longitude'], 
                                           central_toronto['borough'], central_toronto['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centraltoronto)  
    
map_centraltoronto

## Exploring Neighborhoods in Central Toronto

In [123]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'OHYNF34MXXW1L3IMFUH45YWGWUYVNXLGUQ0NXVF12PK1FNB3' # your Foursquare ID
CLIENT_SECRET = 'KBLUEPR5AM0YAAKUNOEXFTO4BLNNM5KLPVHEPN305TR4RGAJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OHYNF34MXXW1L3IMFUH45YWGWUYVNXLGUQ0NXVF12PK1FNB3
CLIENT_SECRET:KBLUEPR5AM0YAAKUNOEXFTO4BLNNM5KLPVHEPN305TR4RGAJ


In [124]:
#Write a function to get the top venues that are in neighbourhoods of Central Toronto within a radius of 500 meters
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                  'neighbourhood latitude', 
                  'neighbourhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

In [125]:
#apply function
centraltoronto_venues = getNearbyVenues(names=central_toronto['neighbourhood'],
                                   latitudes=central_toronto['latitude'],
                                   longitudes=central_toronto['longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [126]:
#check the size of new df
print(centraltoronto_venues.shape)
centraltoronto_venues.head()

(108, 7)


,neighbourhood,neighbourhood latitude,neighbourhood longitude,venue,venue latitude,venue longitude,venue category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service


In [127]:
#let's see how many venue categories there are for all neighbourhoods
centraltoronto_venues.groupby('neighbourhood')['venue category'].count()

neighbourhood
Davisville                                                           33
Davisville North                                                      7
Forest Hill North & West, Forest Hill Road Park                       5
Lawrence Park                                                         4
Moore Park, Summerhill East                                           3
North Toronto West,  Lawrence Park                                   20
Roselawn                                                              3
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park    14
The Annex, North Midtown, Yorkville                                  19
Name: venue category, dtype: int64

In [128]:
#let's see how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(centraltoronto_venues['venue category'].unique())))

There are 59 uniques categories.


## Analyse Neighborhoods in Central Toronto

Let's find out the top five most popular venues in each neighbourhood

In [129]:
# one hot encoding
centraltoronto_onehot = pd.get_dummies(centraltoronto_venues[['venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centraltoronto_onehot['neighbourhood'] = centraltoronto_venues['neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [centraltoronto_onehot.columns[-1]] + list(centraltoronto_onehot.columns[:-1])
centraltoronto_onehot = centraltoronto_onehot[fixed_columns]

centraltoronto_onehot.head()

,neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
centraltoronto_grouped = centraltoronto_onehot.groupby('neighbourhood').mean().reset_index()
centraltoronto_grouped.head()

,neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0,0.0,0.0,0.0,0.000000,0.030303,0.0,0.00,0.060606,0.0,0.0,0.060606,0.0,0.000000,0.090909,0.00,0.030303,0.030303,0.0,0.030303,0.0,0.000000,0.0,0.0,0.030303,0.030303,0.030303,0.060606,0.0,0.0,0.0,0.000000,0.0,0.030303,0.060606,0.0,0.0,0.0,0.0,0.0,0.030303,0.030303,0.090909,0.0,0.030303,0.0,0.090909,0.030303,0.0,0.0,0.0,0.060606,0.00,0.000000,0.030303,0.030303,0.000000,0.0,0.0
1,Davisville North,0.0,0.0,0.0,0.0,0.142857,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.142857,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.142857,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.142857,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000,0.142857,0.0,0.000000,0.0,0.142857,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,"Forest Hill North & West, Forest Hill Road Park",0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.20,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.2,0.0,0.0,0.0,0.0,0.200000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.200000,0.00,0.000000,0.000000,0.000000,0.200000,0.0,0.0
3,Lawrence Park,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,"Moore Park, Summerhill East",0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.333333,0.000000,0.000000,0.333333,0.0,0.0


In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [150]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['neighbourhood'] = centraltoronto_grouped['neighbourhood']

for ind in np.arange(centraltoronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centraltoronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Coffee Shop
1,Davisville North,Department Store,Breakfast Spot,Park,Sandwich Place,Food & Drink Shop
2,"Forest Hill North & West, Forest Hill Road Park",Park,Bus Line,Trail,Sushi Restaurant,Jewelry Store
3,Lawrence Park,Swim School,Bus Line,Park,Dim Sum Restaurant,Yoga Studio
4,"Moore Park, Summerhill East",Trail,Tennis Court,Park,Yoga Studio,Dessert Shop
5,"North Toronto West, Lawrence Park",Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop
6,Roselawn,Home Service,Ice Cream Shop,Garden,Yoga Studio,Gym
7,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,American Restaurant,Restaurant,Fried Chicken Joint,Vietnamese Restaurant
8,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,History Museum,Donut Shop


In the table above we have identified the weighted average of most common venues for Central Toronto neighbourhoods. 

The most common venue is different in each neighbourhood, except for Davisbille and Annex, North Midtown, Yorkville which is sandwich place. There is little repetition, each neighbourhood is quite different in terms of the common venues.

## Cluster Central Toronto neighbourhoods

In [134]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

centraltoronto_grouped_clustering = centraltoronto_grouped.drop(['neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centraltoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 0, 4, 1, 3, 1, 1])

 create a new dataframe that includes the cluster as well as the neighborhood

In [135]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'cluster labels', kmeans.labels_)

centraltoronto_merged = central_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
centraltoronto_merged = centraltoronto_merged.join(neighbourhoods_venues_sorted.set_index('neighbourhood'), on='neighbourhood')

centraltoronto_merged.head() # check the last columns!

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Bus Line,Park,Dim Sum Restaurant,Yoga Studio
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Home Service,Ice Cream Shop,Garden,Yoga Studio,Gym
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Department Store,Breakfast Spot,Park,Sandwich Place,Food & Drink Shop
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Bus Line,Trail,Sushi Restaurant,Jewelry Store
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop


In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centraltoronto_merged['latitude'], centraltoronto_merged['longitude'], 
                                  centraltoronto_merged['neighbourhood'], centraltoronto_merged['cluster labels']):
    label = folium.Popup(str(poi) + ' cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine clusters

In [168]:
# let's check how many neighbourhoods are in each cluster
centraltoronto_merged.groupby('cluster labels')['neighbourhood'].count()

cluster labels
0    1
1    5
2    1
3    1
4    1
Name: neighbourhood, dtype: int64

Basically we have only one cluster which include 5 neighbourhoods, and the rest neighbourhoods represent separate cluster. 

In [158]:
centraltoronto_merged.loc[centraltoronto_merged['cluster labels'] == 0]

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,0,Swim School,Bus Line,Park,Dim Sum Restaurant,Yoga Studio


In [159]:
centraltoronto_merged.loc[centraltoronto_merged['cluster labels'] == 1]

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Department Store,Breakfast Spot,Park,Sandwich Place,Food & Drink Shop
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,History Museum,Donut Shop
6,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Coffee Shop
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,1,Coffee Shop,American Restaurant,Restaurant,Fried Chicken Joint,Vietnamese Restaurant


The cluster 1 is defined by a large number of food places, when comparing to other clusters.

In [160]:
centraltoronto_merged.loc[centraltoronto_merged['cluster labels'] == 2]

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Bus Line,Trail,Sushi Restaurant,Jewelry Store


In [161]:
centraltoronto_merged.loc[centraltoronto_merged['cluster labels'] == 3]

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Home Service,Ice Cream Shop,Garden,Yoga Studio,Gym


In [162]:
centraltoronto_merged.loc[centraltoronto_merged['cluster labels'] == 4]

,postal_code,borough,neighbourhood,latitude,longitude,cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316,4,Trail,Tennis Court,Park,Yoga Studio,Dessert Shop
